In [76]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

In [54]:
url='https://gist.githubusercontent.com/avnreddy95/0d26cf9d9059a388e9034fa902a637d9/raw/6220944aa507e672c3cfa0cb39601d57bff6693f/credit_card.csv'
df = pd.read_csv(url)

In [55]:
df.shape

(690, 16)

In [56]:
df.head(10)

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,1,30.83,0.000,1,1,Industrials,White,1.250,1,1,1,0,ByBirth,202,0,1
1,0,58.67,4.460,1,1,Materials,Black,3.040,1,1,6,0,ByBirth,43,560,1
2,0,24.50,0.500,1,1,Materials,Black,1.500,1,0,0,0,ByBirth,280,824,1
3,1,27.83,1.540,1,1,Industrials,White,3.750,1,1,5,1,ByBirth,100,3,1
4,1,20.17,5.625,1,1,Industrials,White,1.710,1,0,0,0,ByOtherMeans,120,0,1
5,1,32.08,4.000,1,1,CommunicationServices,White,2.500,1,0,0,1,ByBirth,360,0,1
6,1,33.17,1.040,1,1,Transport,Black,6.500,1,0,0,1,ByBirth,164,31285,1
7,0,22.92,11.585,1,1,InformationTechnology,White,0.040,1,0,0,0,ByBirth,80,1349,1
8,1,54.42,0.500,0,0,Financials,Black,3.960,1,0,0,0,ByBirth,180,314,1
9,1,42.50,4.915,0,0,Industrials,White,3.165,1,0,0,1,ByBirth,52,1442,1


In [57]:
df['Approved'].value_counts()

Approved
0    383
1    307
Name: count, dtype: int64

In [58]:
df.drop('Industry', axis=1, inplace=True)

In [59]:
df.describe(),df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          690 non-null    int64  
 1   Age             690 non-null    float64
 2   Debt            690 non-null    float64
 3   Married         690 non-null    int64  
 4   BankCustomer    690 non-null    int64  
 5   Ethnicity       690 non-null    object 
 6   YearsEmployed   690 non-null    float64
 7   PriorDefault    690 non-null    int64  
 8   Employed        690 non-null    int64  
 9   CreditScore     690 non-null    int64  
 10  DriversLicense  690 non-null    int64  
 11  Citizen         690 non-null    object 
 12  ZipCode         690 non-null    int64  
 13  Income          690 non-null    int64  
 14  Approved        690 non-null    int64  
dtypes: float64(3), int64(10), object(2)
memory usage: 81.0+ KB


(           Gender         Age        Debt     Married  BankCustomer  \
 count  690.000000  690.000000  690.000000  690.000000    690.000000   
 mean     0.695652   31.514116    4.758725    0.760870      0.763768   
 std      0.460464   11.860245    4.978163    0.426862      0.425074   
 min      0.000000   13.750000    0.000000    0.000000      0.000000   
 25%      0.000000   22.670000    1.000000    1.000000      1.000000   
 50%      1.000000   28.460000    2.750000    1.000000      1.000000   
 75%      1.000000   37.707500    7.207500    1.000000      1.000000   
 max      1.000000   80.250000   28.000000    1.000000      1.000000   
 
        YearsEmployed  PriorDefault    Employed  CreditScore  DriversLicense  \
 count     690.000000    690.000000  690.000000    690.00000      690.000000   
 mean        2.223406      0.523188    0.427536      2.40000        0.457971   
 std         3.346513      0.499824    0.495080      4.86294        0.498592   
 min         0.000000      0.0

In [64]:
label={}
for col in df.columns:
    if df[col].dtype=='object':
        le=LabelEncoder()
        df[col]=le.fit_transform(df[col])
        label[col]=le
       

In [67]:
x=df.drop('Approved', axis=1)
y=df['Approved']

In [68]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [69]:
#train the model
model=RandomForestClassifier(random_state=42,class_weight='balanced')
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [74]:
#evaluate the model
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print(roc_auc_score(y_test, model.predict_proba(x_test)[:, 1]))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87        68
           1       0.89      0.83      0.86        70

    accuracy                           0.86       138
   macro avg       0.86      0.86      0.86       138
weighted avg       0.86      0.86      0.86       138

ROC AUC Score: 0.8628151260504202
0.9097689075630253


In [79]:
# apply SMOTE
smote= SMOTE(random_state=42)
x_resampled,y_resampled=smote.fit_resample(x_train,y_train)

In [80]:
#display new class distribution

print('class distribution after SMOTE:',pd.Series(y_resampled).value_counts())

class distribution after SMOTE: Approved
0    315
1    315
Name: count, dtype: int64


In [81]:
# train random forest model on resampled data
model_resampled=RandomForestClassifier(random_state=42,class_weight='balanced')
model_resampled.fit(x_resampled,y_resampled)
# make predictions on test set
y_pred_resampled=model_resampled.predict(x_test)
# evaluate the model
print(classification_report(y_test, y_pred_resampled))
print("ROC AUC Score after SMOTE:", roc_auc_score(y_test, y_pred_resampled))
print(roc_auc_score(y_test, model_resampled.predict_proba(x_test)[:, 1]))


              precision    recall  f1-score   support

           0       0.85      0.85      0.85        68
           1       0.86      0.86      0.86        70

    accuracy                           0.86       138
   macro avg       0.86      0.86      0.86       138
weighted avg       0.86      0.86      0.86       138

ROC AUC Score after SMOTE: 0.8550420168067226
0.9067226890756303
